# Snaike game with neural nets

## Importing modules

In [1]:
from Snake import Snake
from body import body
from head import head
from food import food
from SnakeTools import *
from SnakeRender import *
from pygame.locals import *
from SnakeAi import *
import pygame,sys
import pickle
import tensorflow as tf
import matplotlib.pyplot as plt

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
#colors
WHITE = (255, 255, 255)
BLACK=(0,0,0)

In [3]:
'''
#reset files

pickle.dump( [], open( "features.fer", "wb" ) )    
pickle.dump( [], open( "tags.fer", "wb" ) )
'''

'\n#reset files\n\npickle.dump( [], open( "features.fer", "wb" ) )    \npickle.dump( [], open( "tags.fer", "wb" ) )\n'

In [4]:
#init
print("Initializing game")
pygame.init()
dim=300
displaysurf=pygame.display.set_mode((dim,dim))
pygame.display.set_caption('Snaike')
fpsClock = pygame.time.Clock()
FPS = 120
framecount=0


#emilia variables
emiliaImg = pygame.image.load('blue.png')
emiliaSnake = Snake(int(dim/2)+100,int(dim/2)+100,emiliaImg,ori='u')
emiliaSnake.createWalls(dim)

#subaro
subaroImg=pygame.image.load('blue.png')
emiliaSnake.bodyImg=subaroImg

#game over
gameoverImg=pygame.image.load('gameover.jpg')

#food
foodImg=pygame.image.load('hamb.png')
foood=newFood(foodImg,dim)

#orientation
lastorientation=emiliaSnake.orientation
orientation=emiliaSnake.orientation
gameOverFlag=False



Initializing game


In [5]:
#Neural net
print("Opening config files")
#training features and tags
features=pickle.load( open( "features.fer", "rb" ) )
tags=pickle.load( open("tags.fer","rb"))
print("Features: ", len(features))

'''
print("Training model ...")
featz=len(features[0])
outz=3
l1_neurons=30

#Neural net config
x=tf.placeholder(tf.float32,shape=[None,featz], name='x')
l1=fully_connected_layer(x,featz,l1_neurons)
l2=fully_connected_layer(l1,l1_neurons,outz,relu=False)
softmax=tf.nn.softmax(l2)
y_pred_cls=tf.argmax(softmax,axis=1)
y_true = tf.placeholder(tf.float32, shape=[None, 3], name='y_true')
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=l2,labels=y_true)
cost = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

#Accuracy measures
y_true_cls = tf.argmax(y_true, axis=1)
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
'''

Opening config files
Features:  4257
Test-split
Smoting
Features:  3831


'\nprint("Training model ...")\nfeatz=len(features[0])\noutz=3\nl1_neurons=30\n\n#Neural net config\nx=tf.placeholder(tf.float32,shape=[None,featz], name=\'x\')\nl1=fully_connected_layer(x,featz,l1_neurons)\nl2=fully_connected_layer(l1,l1_neurons,outz,relu=False)\nsoftmax=tf.nn.softmax(l2)\ny_pred_cls=tf.argmax(softmax,axis=1)\ny_true = tf.placeholder(tf.float32, shape=[None, 3], name=\'y_true\')\ncross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=l2,labels=y_true)\ncost = tf.reduce_mean(cross_entropy)\noptimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)\n\n#Accuracy measures\ny_true_cls = tf.argmax(y_true, axis=1)\ncorrect_prediction = tf.equal(y_pred_cls, y_true_cls)\naccuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))\n'

In [6]:
#Session
session = tf.Session()
session.run(tf.global_variables_initializer())

In [7]:
'''
#Optimization function
sampleGenerator=SampleGenerator(X_train,y_train)

'''

'\n#Optimization function\nsampleGenerator=SampleGenerator(X_train,y_train)\n\n'

In [8]:
'''
#Running the graph
optimize(7500)
'''

'\n#Running the graph\noptimize(7500)\n'

In [9]:
'''
pred_dict={x:X_test}
predictions=session.run(y_pred_cls,feed_dict=pred_dict)

print(classification_report(y_test,predictions))
'''

'\npred_dict={x:X_test}\npredictions=session.run(y_pred_cls,feed_dict=pred_dict)\n\nprint(classification_report(y_test,predictions))\n'

In [10]:
#AI move
def aiMove(emiliaSnake,foood,orientation,framecount):
        
    if(framecount==0):
        atr1,atr2,atr3=emiliaSnake.getAtr()
        ang,hip= foodPos(emiliaSnake,foood)
            
        run_dict={x:[[atr1,atr2,atr3,ang]]}
        mv=session.run(y_pred_cls,feed_dict=run_dict)-1
        print(mv)
        if(orientation=='u' and mv==1):
            orientation='r'
        elif(orientation=='r' and mv==-1):
            orientation='u'
        else:
            orientation=numbOri(mv+oriNumb(orientation))
        print(orientation)
    
    return orientation

In [11]:
#game loop
print("game starts")
while True:   
    displaysurf.fill(WHITE)
    
    for event in pygame.event.get():
        if event.type== QUIT:
            pickle.dump( features, open( "features.fer", "wb" ) )    
            pickle.dump( tags, open( "tags.fer", "wb" ) )
            pygame.quit()
            sys.exit()
        elif event.type==KEYDOWN:
            if event.key==K_UP and orientation!='d':
                orientation='u'
            elif event.key==K_DOWN and orientation!='u':
                orientation='d'
            elif event.key==K_RIGHT and orientation!='l':
                orientation='r'
            elif event.key==K_LEFT and orientation!='r':
                orientation='l'
            elif event.key==K_f:
                emiliaSnake = Snake(int(dim/2),int(dim/2),emiliaImg)
                emiliaSnake.createWalls(dim)
                emiliaSnake.bodyImg=subaroImg
                gameOverFlag=False
            elif event.key==K_n:
                pickle.dump( features, open( "features.fer", "wb" ) )
                pickle.dump( tags, open( "tags.fer", "wb" ) )
                pygame.quit()
                sys.exit()
    
    framecount=(framecount+1)%30
    
    atr1,atr2,atr3,ang,hip=0,0,0,0,0
    #emilia motion
    if(emiliaSnake.isAlive()):
        
        '''
        #AI move
        if(True):
            atr1,atr2,atr3=emiliaSnake.getAtr()
            ang,hip= foodPos(emiliaSnake,foood)
            run_feat=[[int(atr1/emiliaSnake.head.image.get_width()),int(atr2/emiliaSnake.head.image.get_width()),int(atr3/emiliaSnake.head.image.get_width()),int(ang)]]
            run_dict={x:run_feat}
            mv=session.run(y_pred_cls,feed_dict=run_dict)-1
            if(orientation=='u' and mv==1):
                orientation='r'
            elif(orientation=='r' and mv==-1):
                orientation='u'
            else:
                orientation=numbOri(mv+oriNumb(orientation))
        '''
        emiliaSnake.move(orientation)
    
    if(not emiliaSnake.isAlive()):
        gameOverFlag=True
        
        displaysurf.blit(gameoverImg, (int((dim-gameoverImg.get_width())/2),(int((dim-gameoverImg.get_height())/2)-50)))
        pygame.display.update()

    if(emiliaSnake.eaten(foood)):
        
        foood=newFood(foodImg,dim)
        emiliaSnake.addBlock()
    
    if(not gameOverFlag):
        
        if(framecount==0 or lastorientation!=orientation):
            
            atr1,atr2,atr3=emiliaSnake.getAtr()
            ang,hip= foodPos(emiliaSnake,foood)
            
            features.append([atr1,atr2,atr3,ang,hip,oriNumb(orientation)])
            rio=calcTag(lastorientation,orientation)
            tags.append(rio)
            lastorientation=orientation
        
        renderSnake(displaysurf,emiliaSnake)
        renderFood(displaysurf,foood)
        pygame.display.update()
        
    fpsClock.tick(FPS)
    

            

game starts


SystemExit: 

C:\Users\Fernando\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
xp=[int(x[1])for x in X_train]
yp=[int(y[0])for y in X_train]

ax.scatter(xp, yp, y_train, c='b', marker='o')
plt.show()

In [ ]:
from sklearn.manifold import TSNE
from mpl_toolkits.mplot3d import Axes3D
features=np.array(features)
nf=[]
nt=[]
print('before')
print(len(features))
for i in range(len(features)):
    ff=features[i]
    if(len(ff)==4):
        nf.append(ff)
        nt.append(tags[i])
print('after')
print(len(nf))
ff = TSNE(n_components=2).fit_transform(nf)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
x=[j[0] for j in ff]
y=[j[1] for j in ff]
print(len(nt), len(nf))
ax.scatter(x, y, nt, c='b', marker='o')
plt.show()